In [ ]:
import gurobi.GRB;
import gurobi.GRBEnv;
import gurobi.GRBException;
import gurobi.GRBLinExpr;
import gurobi.GRBModel;
import gurobi.GRBVar;

/**
 *
 * @author SEMS RA-02
 */
public class _2018concretedeliverynew {

    /**
     * @param args the command line arguments
     * @throws gurobi.GRBException
     */
    public static void main(String[] args) throws GRBException {
        //the application logic of the problem
        try {

            // Defining Sets

            //Set of Vehicles
            String Vehicles[]
                    = new String[]{"K1", "K2", "K3", "K4"};
            double VehicleCapacity[] = new double[]{5, 5, 5, 5};

            //Demand
            int Demand[] = new int[]{20, 5, 10};
            int CustomerNode[] = new int[]{4, 5, 6};
            int MaxNumDeliveryC[] = new int[]{4, 1, 2};
            //min time lag customer
            double mintlC[] = new double[]{10.0 / 60, 0.0 / 60, 15.0 / 60};
            //max time lag customer
            double maxtlC[] = new double[]{20.0/60, 20.0/60, 15.0/60};

            int DepotNode[] = new int[]{0, 2};
            int nDepotNode = DepotNode.length;
            //min time lag depot
            double mintlD[] = new double[]{30.0/60, 20.0/60};
            int MaxNumDeliveryD[] = new int[]{7, 10};



            //Nodes
            int NodeSet[] = new int[]{0, 1, 2, 3, 4, 5, 6};
            int nNodeSet = NodeSet.length;

            // NodeSetC or customer nodes or C^G = {C1_1, C1_2, C1_3, C1_4, C2_1, C3_1, C3_2}
            //                               C^G = {   0,    1,    2,    3,    4,    5,    6}
            // NodeSetD or depot nodes or D^G = {D1_1, D1_2, D1_3, D1_4, D1_5, D1_6, D1_7, D2_1, D2_2, D2_3, D2_4, D2_5, D2_6, D2_7, D2_8, D2_9, D2_10}
            //                            D^G = {   7,    8,    9,   10,   11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,    23}
            // NodeSetO = {O1, O2, O3, O4}
            //          = {24, 25, 26, 27}
            // NodeSetF = {F1, F2, F3, F4}
            //          = {28, 29, 30, 31}
            // AllNodeSet or node set V = C^G U D^G U O U F
            // AllNodeSet or node set V =            {C1_1, C1_2, C1_3, C1_4, C2_1, C3_1, C3_2, D1_1, D1_2, D1_3, D1_4, D1_5, D1_6, D1_7, D2_1, D2_2, D2_3, D2_4, D2_5, D2_6, D2_7, D2_8, D2_9, D2_10, O1, O2, O3, O4, F1, F2, F3, F4}
            // AllNodeSetIndex or node set V =       {   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,    23, 24, 25, 26, 27, 28, 29, 30, 31}
            int AllNodeSetIndex[] = new int[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}; //revised
            int AllNodeSet[] = new int[]{4, 4, 4, 4, 5, 6, 6, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 0, 0, 3, 2, 0, 0}; //revised
            int AllNodeSetSequence[] = new int[]{0, 1, 2, 3, 0, 0, 1, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, -2, -2, -2, -2, -1, -1, -1, -1}; //revised
            double AllNodeServiceTime[] = new double[]{5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60};

            int nAllNodeSetIndex = AllNodeSetIndex.length;


            //int NodeSetTo[] = new int[]         {0, 1, 2, 3, 4, 5, 6};
            int NodeSetD[] = new int[]{7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}; //revised
            //nah ini di constraint (10 dan 11) sepertinya digabung time windowsnya, jadi bagaimana ya pak
            double timeStartD[] = new double[]{7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25};
            double timeEndD[] = new double[]{10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25};
            int NodeSetC[] = new int[]{0, 1, 2, 3, 4, 5, 6}; //revised
            //time windows [a,b] - gak ngerti
            double timeStartC[] = new double[]{8.0, 8.0, 8.0, 8.0, 8.5, 8.5, 8.5,};
            //first delivery deadline ==> diasumsikan 30 menit setelah time windows start
            double FirstDelivery[] = new double[]{10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0};
            double timeEndC[] = new double[]{9.0, 9.0, 9.0, 9.0, 10.0, 10.0, 10.0};
            int NodeSetCD[] = new int[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}; //revised
            int NodeSetO[] = new int[]{24, 25, 26, 27}; //revised
            int NodeSetF[] = new int[]{28, 29, 30, 31}; //revised

            int nNodeSetC = NodeSetC.length;
            int nNodeSetD = NodeSetD.length;
            int nNodeSetCD = NodeSetCD.length;

            //Edges and Nodes and Costs
            // Edges Set = {O_F, O_D, D_C, C_D, D_F} while each of this edge is assigned to each vehicle. so that
            // Edges Set K or AllEdgeSet = {O_F_K1, O_F_K2, O_F_K3, O_F_K4, O_D1_1_K1, O_D1_1_K2, O_D1_1_K3, O_D1_1_K4, O_D2_1_K1, O_D2_2_K1, O_D3_3_K1, O_D_4, D_C_1, D_C_2, D_C_3, D_C_4, C_D_1, C_D_2, C_D_3, C_D_4, D_F_1, D_F_2, D_F_3, D_F_4}
            //                AllEdgeSet = {    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,    10,    11,    12,    13,    14,    15,    16,    17,    18,    19}
            int AllEdgeSet[] = new int[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337};
            int NodesFrom[] = new int[]{24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6};
            int NodesTo[] = new int[]{28, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 29, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 30, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 31, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 28, 29, 30, 31, 28, 29, 30, 31, 28, 29, 30, 31, 28, 29, 30, 31, 28, 29, 30, 31, 28, 29, 30, 31, 28, 29, 30, 31};
            int CostEdge[] = new int[]{0, 12, 12, 12, 12, 12, 12, 12, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 20, 20, 20,